In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import healpy as hp
import scipy
from numba import jit, njit, prange, set_num_threads
from tqdm.notebook import tqdm
from helper_funcs import *

In [4]:
def sort_alms(alms, lmax):
    '''
    Sorts healpix alm's by \ell instead of m given a fortran90 array output from hp.map2alm.

    Parameters
    ----------
    alms : fortran90 healpix alm array from hp.map2alm
    num_ls : number of l's in alms array
    
    Returns
    ----------
    sorted_alms : alm dictionary keyed by ell values with numpy arrays consisting of the corresponding m values.
    '''
    start = 0
    sorted_alms = {}

    for l in range(lmax):
        sorted_alms[l] = np.zeros(2*l+1, dtype=np.cdouble)

    for m in range(lmax):
        num_ms = lmax - m
        ms = alms[start:num_ms + start]
        start = num_ms + start
        m_sign = (-1)**m
        for l in range(num_ms):
            idx = m + l
            sorted_alms[idx][m] = ms[l]
            if m != 0:
                sorted_alms[idx][-m] = m_sign * np.conj(ms[l])
    
    return sorted_alms

In [5]:
lmax = 1000
ells = np.arange(lmax+1)
cls = np.zeros_like(ells, dtype='float')

for l in ells[1:]:
    cls[l] = (l+0.0)**(-3.)

theory_map, unsorted_alms = hp.sphtfunc.synfast(cls=cls, nside=1024, lmax=lmax, alm=True)

In [6]:
sorted_alms = sort_alms(unsorted_alms, len(cls))

In [171]:
def unsort_alms(sorted_alms):
    """
    Unsorts a sorted_alm dictionary from sorted_alms into a Fortran90
    array that can be processed by HEALPix.

    Inputs:
    sorted_alms (dict[ndarray]) : sorted alms from sort_alms

    Returns:
    (ndarray) : unsorted alms for HEALPix
    """
    lmax = max(sorted_alms.keys())
    unsorted_alms = np.zeros(np.sum(2 * np.arange(0, lmax + 1) + 1), dtype=np.complex128)

    start = 0
    for m in range(lmax + 1):
        for ell in range(m, lmax + 1):
            unsorted_alms[start + ell + m] = sorted_alms[ell][m]
        start += lmax
    
    return unsorted_alms


In [172]:
test = unsort_alms(sorted_alms)

In [200]:
print(sorted_alms[3][5])

(-0.06867491506315603-0.11835288965775595j)


In [180]:
val = 1004
print(unsorted_alms[val])
print(test[val])

(0.008047815896582818-0.06274007698141973j)
(0.03624870027048562-0.08102201438388737j)


We divide the $\ell$-range $\left[\ell_{\min }, \ell_{\max }\right]$ into subintervals denoted by $\Delta_i=\left[\ell_i, \ell_{i+1}-1\right]$ where $i=0, \ldots,\left(N_{\text {bins }}-1\right)$ and $\ell_{N_{\text {bins }}}=\ell_{\max }+1$, so that the filtered maps are:

\begin{equation*}
M_i^p(\Omega)=\sum_{\ell \in \Delta_i} \sum_{m=-\ell}^{+\ell} a_{\ell m}^p Y_{\ell m}(\hat{\Omega}) \tag{2.5}
\end{equation*}

and we use these instead of $M_{\ell}^p$ in the expression for the bispectrum (2.5). The binned bispectrum is:

\begin{equation*}
B_{i_1 i_2 i_3}^{p_1 p_2 p_3, \mathrm{obs}}=\frac{1}{\Xi_{i_1 i_2 i_3}} \int d \hat{\Omega} M_{i_1}^{p_1, \mathrm{obs}}(\hat{\Omega}) M_{i_2}^{p_2, \mathrm{obs}}(\hat{\Omega}) M_{i_3}^{p_3, \mathrm{obs}}(\hat{\Omega}) \tag{2.6}
\end{equation*}

where $\Xi_{i_1 i_2 i_3}$ is the number of $\ell$ triplets within the $\left(i_1, i_2, i_3\right)$ bin triplet satisfying the triangle inequality and parity condition selection rule. Because of this normalization factor, $B_{i_1 i_2 i_3}^{p_1 p_2 p_3}$ may be considered an average over all valid $B_{\ell_1 \ell_2 \ell_3}^{p_1 p_2 p_3}$ inside the bin triplet.

In [17]:
test = np.zeros_like(alms)

In [9]:
lmin, lmax = 2, 10

ibin = np.arange(lmin, lmax + 1)

filtered_alms = np.zeros_like(alms)

for l in ibin:
    for m in range(-l, l + 1):
        filtered_alms

TypeError: alm2map() missing 2 required positional arguments: 'alms' and 'nside'

In [98]:
@njit(parallel=True)
def count_valid_configs(i1, i2, i3, num_threads=16):
    """
    Counts the number of valid ell-triplet configurations in a single bin.
    Validity is determined as satisfying the parity condition selection
    rule and the triangle inequality.

    Inputs:
    i1 (int) : bin1
    i2 (int) : bin2
    i3 (int) : bin3
    num_threads (int) : number of threads to parallelize on

    Returns:
    (int) : number of valid configurations of ell-triplets.
    """

    set_num_threads(num_threads)

    configs = 0

    for l1 in prange(i1 + 1):
        for l2 in range(i2 + 1):
            for l3 in range(i3 + 1):
                if check_valid_triangle(l1, l2, l3):
                    configs += 1
    
    return configs

In [96]:
count_valid_configs(100, 100, 100)

45526

In [94]:
@njit
def check_valid_triangle(l1, l2, l3):
    """
    Checks if a set of l1, l2, l3's form a valid triangle, which can be
    confirmed if the l's satisfy the following three metrics:
    1. bispectrum is symmetric under any l1, l2, l3 permutations
    2. even parity selection rule
    3. triangle inequality

    Inputs:
    l1 (int) : l1
    l2 (int) : l2
    l3 (int) : l3
    
    Returns:
    (bool) : returns True if triangle is valid, False otherwise
    """
    permutations = l1 <= l2 <= l3
    even_parity = (l1 + l2 + l3) % 2 == 0
    tri_inequality = np.abs(l1 - l2) <= l3 <= l1 + l2

    return permutations and even_parity and tri_inequality


In [39]:
# def P(l, m, x):
# 	pmm = 1.0
# 	if m > 0:
# 		somx2 = np.sqrt((1.0 - x) * (1.0 + x))
# 		fact = 1.0
# 		for i in range(1, m + 1):
# 			pmm *= (-fact) * somx2
# 			fact += 2.0
	
# 	if l == m:
# 		return pmm * np.ones(x.shape)
	
# 	pmmp1 = x * (2.0*m+1.0) * pmm
	
# 	if l == m + 1:
# 		return pmmp1
	
# 	pll = np.zeros(x.shape)
# 	for ll in range(m + 2, l + 1):
# 		pll = ((2.0 * ll - 1.0) * x * pmmp1 - (ll + m - 1.0) * pmm) / (ll - m)
# 		pmm = pmmp1
# 		pmmp1 = pll
	
# 	return pll

In [ ]:
# def ylm_norm_factor(l, m):
#     """
#     Finds the normalization factor for the spherical harmonic function
#     a given ell and m.

#     Inputs:
#     l : (int) ell-value
#     m : (int) m-value

#     Returns:
#     (int) : normalization factor for spherical harmonic function
#     """
#     return np.sqrt((2 * l + 1)/(4 * np.pi) \
#         * ((math.factorial(l - m))/(math.factorial(l + m))))

# def ylm(omega, l, m):
#     """
#     The spherical harmonic functions.

#     Inputs:
#     omega : (var) solid-angle omega
#     l : (int) ell-value
#     m : (int) m-value

#     Returns:
#     (func) : spherical harmonic function for a given ell, m input.
#     """